Pyspark installation

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install pyspark

Get files from URL to local storage

In [ ]:
#from google.colab import files
#files.upload()

In [ ]:
url = "http://stateoftheunion.onetwothree.net/texts/index.html"
headers = {'User-Agent': "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7"}
import urllib.request
try: response=urllib.request.urlopen(urllib.request.Request(url, None, headers)).read()
except urllib.error.URLError as e: print(e)

In [ ]:
import re, os
name=re.findall(r'"(.*?)\"',''.join(re.findall(r'<li>(.*?)</li>',str(response))[5:]))
os.makedirs('files')
for i in name:
    try: fetch=urllib.request.urlopen(urllib.request.Request(('http://stateoftheunion.onetwothree.net/texts/'+i),None,headers)).read()
    except urllib.error.URLError as e: print(e)
    #with open(os.path.join("C:\\Users\\Pratik\\Desktop\\CS657\\Assignment 1\\Files\\",i.split('.')[0]), 'w') as file: file.write(str(fetch))
    with open(os.path.join("./files",i.split('.')[0]), 'w') as file: file.write(str(fetch))

Start spark


In [ ]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("WordCount")
sc = SparkContext(conf = conf)

In [ ]:
rdd = sc.wholeTextFiles('./files/*')

Preprocessing

In [ ]:
rdd_1=rdd.map(lambda x: (x[0].split('/')[-1],x[1].split("</h3>")[1])).map(lambda x: (x[0],x[1].split('<div class="textNav">')[0].lower()))

In [ ]:
elim=[str(x)+'.' for x in range(10)]+[str(x) for x in range(10)]+['\\n','\\t','<p>','</p>','<br>',',', '!','?','#',';',':','-','_','+','=','#','%','&','*','(',')','[',']','{','}','"','|',"\\'",'\\',"'",'`','$']
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
elim+=list(map(lambda x: ' '+x+' ',stopwords.words('english')))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def clean(x):
    for i in elim: x=(x[0],x[1].replace(i,' '))
    return (x[0],' '.join(x[1].split()))
rdd_2=rdd_1.map(clean).map(lambda x: (x[0],x[1].replace('.',' . ')))       #with . retained
rdd_3=rdd_2.map(lambda x: (x[0],x[1].replace(' . ','')))

**Part 1**

Average use of every word over all the years

In [ ]:
years=rdd_3.map(lambda x: int(x[0][:4]))
total_years=years.max()-years.min()
avg_allyears=rdd_3.map(lambda x: x[1].split()).flatMap(lambda x:x).map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y).map(lambda x: (x[0],x[1]/total_years))
#avg_allyears.take(20)
#avg_allyears.saveAsTextFile('out_1_avg_allyears.txt')

Maximum and Minimum times word appears in all the addresses

In [ ]:
ads_word=rdd_3.map(lambda x: [((x[0],y),1) for y in x[1].split()]).flatMap(lambda x: x).reduceByKey(lambda x,y: x+y)

In [ ]:
word_vals=ads_word.map(lambda x: (x[0][1],x[1])).groupByKey().map(lambda x: (x[0],max(x[1]),min(x[1])))
#word_vals.take(20)
#word_vals.saveAsTextFile('out_2_word_vals.txt')

Starting 1992, average and std deviation of times word appears in window of 4 years

In [ ]:
word_years=ads_word.filter(lambda x: int(x[0][0])>19920000).map(lambda x: ((x[0][0][:4],x[0][1]),x[1])).reduceByKey(lambda x,y: x+y)

In [ ]:
word_range=word_years.map(lambda x: ((str(int(x[0][0])-((int(x[0][0])%4))),x[0][1]),x[1])).reduceByKey(lambda x,y: x+y)
rdd_4=word_range.map(lambda x: (x[0][1],(x[0][0]+'-'+str(int(x[0][0])+4),x[1])))
word_4_avg=rdd_4.map(lambda x: (x[0],(x[1][0],x[1][1]/4)))
word_4yrs_avg=word_4_avg.groupByKey().mapValues(list)
import math
year_val=math.ceil((years.max()-1992)/4)
word_4yrs_std=rdd_4.map(lambda x: (x[0],x[1][1])).groupByKey().mapValues(list).map(lambda x: (x[0],(sum([(y-(sum(x[1])/year_val))**2 for y in x[1]])/year_val)**0.5))
#len of x[1] is variable but range of number of years are 8 (1992-2024)

In [ ]:
word_4yrs_avg_std=word_4yrs_avg.join(word_4yrs_std)
#word_4yrs_avg_std.take(20)
#word_4yrs_avg_std.coalesce(1).saveAsTextFile('out_3_word_4yrs_avg&std.txt')

Words that appear in year following window with frequency exceeding 2 std-devs

In [ ]:
word_std=word_4yrs_avg_std.map(lambda x: (x[0],x[1][1]))
word_nextyr=word_4_avg.map(lambda x: (x[0],(int(x[1][0].split('-')[1])+1,x[1][1])))
word_threshold=word_nextyr.join(word_std).map(lambda x: ((x[0],str(x[1][0][0])),x[1][0][1]+2*x[1][1]))
word_years_remapped=word_years.map(lambda x: ((x[0][1],x[0][0]),x[1]))

In [ ]:
words_freq2std=word_threshold.join(word_years_remapped).map(lambda x: (True if x[1][0]<x[1][1] else False,x[0])).filter(lambda x: x[0]).map(lambda x: x[1])
word_list=words_freq2std.map(lambda x: x[0]).distinct()
#word_list.take(20)
#word_list.coalesce(1).saveAsTextFile('out_4_words_freq_exceed_2std.txt')

**Part 2**

In [ ]:
#using rdd_2 which was cleaned of all punctuations and tags but only . retained
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('WordCount_partB').getOrCreate()
df_1=rdd_2.toDF(['date','speech'])

frequent co-occurances of words (>5)

In [ ]:
from pyspark.sql.functions import split, explode, trim, col
sentences=df_1.withColumn("speech",explode(split(col("speech"),' . '))).withColumnRenamed('speech','sentences').drop('date').dropDuplicates()
sentences=sentences.filter(sentences.sentences!='')
sentences=sentences.filter(sentences.sentences!=' ')
sent_word=sentences.withColumn("word_list", sentences["sentences"]).withColumn('word_list',explode(split(trim(col("word_list")),' ')))
words=sent_word.filter(sent_word.word_list!='')

In [ ]:
df_copy1=words.alias('df_copy1').withColumnRenamed('word_list','words1')
df_copy2=words.alias('df_copy2').withColumnRenamed('word_list','words2').withColumnRenamed('sentences','sent_dup')

In [ ]:
comb=df_copy1.join(df_copy2,[df_copy1.sentences==df_copy2.sent_dup,df_copy1.words1<df_copy2.words2],'inner').drop('sent_dup')

In [ ]:
import pyspark.sql.functions as func
all_combinations=comb.groupBy(['sentences','words1','words2']).count()
combinations=all_combinations.where(func.col('count')>5).drop('sentences')
#combinations.drop('count').show()
#combinations.coalesce(1).write.csv("out_5_word_combinations_exceed_5")

Condiitional Probability > 0.8

In [ ]:
no_sen=sentences.count()
from pyspark.sql.functions import size
#countallwords=sentences.withColumn("totalWordCount",sentences["sentences"]).withColumn('totalWordCount',size(split(trim(col('totalWordCount')),' ')))
countwords=sentences.withColumn("UniqueWords",sentences["sentences"]).withColumn('UniqueWords',explode(split(trim(col('UniqueWords')),' '))).groupBy(['sentences','UniqueWords']).count()
occ_words=countwords.select('UniqueWords').groupBy('UniqueWords').count()
word_probability=occ_words.withColumn('probability',col('count')/no_sen).drop('count')

In [ ]:
word1_prob=all_combinations.join(word_probability,[all_combinations.words1==word_probability.UniqueWords],'inner').drop('UniqueWords').withColumnRenamed('probability','prob(word1)')
indiv_prob=word1_prob.join(word_probability,[word1_prob.words2==word_probability.UniqueWords],'inner').drop('UniqueWords').withColumnRenamed('probability','prob(word2)')
combined=all_combinations.select('words1','words2').groupBy(['words1','words2']).count()
comb_prob=combined.withColumn('prob(AB)',col('count')/no_sen).drop('count').withColumnRenamed('words1','w1').withColumnRenamed('words2','w2')

In [ ]:
final=indiv_prob.join(comb_prob,[indiv_prob.words1==comb_prob.w1,indiv_prob.words2==comb_prob.w2],'inner').drop('sentences','count','w1','w2').dropDuplicates()
final=final.withColumn('P(A/B)',col('prob(AB)')/col('prob(word2)')).withColumn('P(B/A)',col('prob(AB)')/col('prob(word1)'))

In [ ]:
prob_fil1=final.filter(col('P(A/B)')>0.8)
prob_fil2=final.filter(col('P(B/A)')>0.8)
prob_filter=prob_fil1.union(prob_fil2)
#prob_filter.show()
#prob_filter.coalesce(1).write.csv("out_6_conditional_probability_0.8")

In [ ]:
lift=final.withColumn('Lift',col('prob(AB)')/(col('prob(word1)')*col('prob(word2)')))
lift_filter=lift.filter(col('Lift')>2.5)
#lift_filter.show()
#lift_filter.coalesce(1).write.csv("out_7_lift_2.5greater")